In [1]:
!pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to c:\users\flavio\appdata\local\temp\pip-req-build-7fafrhj3
  Resolved https://github.com/deepset-ai/haystack.git to commit c38943721fbd702367a8934cb660a72b3143eb86
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for canals==0.4.0 from https://files.pythonhosted.org/packages/b2/38/d91124721f0a6885fafee64352cd6dd6ba1450d6dd86258a59c551424f05/canals-0.4.0-py3-none-any.whl.metadata
  Using cached canals-0.4.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached lazy_imports-0.3.1-py3-none-any.whl (12 kB)
  Obtaining dependency information for prompthub-py==4.0.0 from https://files.pythonhosted.org/packages/27/5f/8c4939e290ff93a

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git 'C:\Users\Flavio\AppData\Local\Temp\pip-req-build-7fafrhj3'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
farm 0.8.0 requires torch<1.9,>1.5, but you have torch 2.0.1+cu117 which is incompatible.
farm 0.8.0 requires transformers==4.6.1, but you have transformers 4.31.0 which is incompatible.


In [1]:
from pprint import pprint
from typing import Dict, Union, Any, List, Optional, Tuple

from haystack import MultiLabel, Document
from haystack.pipelines import Pipeline
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever
from haystack.nodes.base import BaseComponent
import pandas as pd

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
import time

In [2]:
caminho_query = "dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv"

In [3]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

# Base de dados câmara dos deputados

In [4]:
df_assunto= pd.read_table(caminho_query, delimiter=";")

In [5]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [6]:
def verificar(y,top_n):
    if y in top_n:
        return 1
    return 0

In [7]:
def avaliacaoRecall(isPreprocess,top_k=20):
    quant_encontrado=0
    quant_relevante =0
    antes = time.time()
    for l,x in zip(y,X):
        
        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)                   
    
    
        top_n_stem_l = pipeline.run(query=tokenized_query3,params={"Retriever": {"top_k": top_k}})
    

        top_n = [top_n_stem_l['documents'][d].meta['name'].strip() for d in range(len(top_n_stem_l['documents']))]              #L
    
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    
    recall = quant_encontrado / quant_relevante
    depois = time.time()
    duracao = depois - antes
    print("Recall: "+str(recall))
    print("Duração: %f" %(duracao))

In [8]:
def avaliacaoRR(isPreprocess,top_k=20):
    
    quant_encontrado=0
    
    for l,x in zip(y,X):
        
        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)                   
    
        top_n_stem_l = pipeline.run(query=tokenized_query3,params={"Retriever": {"top_k": top_k}})
        top_n = [top_n_stem_l['documents'][d].meta['name'].strip() for d in range(len(top_n_stem_l['documents']))]              #L
        if l.strip() == top_n[0]:
            quant_encontrado+=1
    rr = quant_encontrado / 295
    print("RR: %f" %(rr))
        
        

In [9]:
def avaliacaoPrecision(isPreprocess,top_k=20):
    soma=0
    for l,x in zip(y,X):
        quant_encontrado=0
        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)                   
       
        top_n_stem_l = pipeline.run(query=tokenized_query3,params={"Retriever": {"top_k": top_k}})
        top_n = [top_n_stem_l['documents'][d].meta['name'].strip() for d in range(len(top_n_stem_l['documents']))]              #L
        
        quant_encontrado=verificar(l,top_n)
        
        soma+=(quant_encontrado/top_k)
        
    rr = soma / 295
    print("Precision: %f" %(rr))        

In [10]:
def avaliacaoMAP(isPreprocess,top_k=20):
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):
        
        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)                   
    
        top_n_stem_l = pipeline.run(query=tokenized_query3,params={"Retriever": {"top_k": top_k}})
        top_n = [top_n_stem_l['documents'][d].meta['name'].strip() for d in range(len(top_n_stem_l['documents']))]              #L
        
        l_v.append(l)
        labels_nsL.append(top_n)
        
    denominador = 0
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP@ %d: %f" % (top_k,MAP))

# ImgTeorPDF

# Pré processamento

## 1- Sem pré processamento

In [11]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="sem_preprocessamento_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [12]:
retriever = BM25Retriever(document_store=document_store)

In [13]:
pipeline = Pipeline()

In [14]:
pipeline.add_node(component=retriever, name="Retriever",inputs=['Query'])

## Avaliação

### Recall

In [15]:
avaliacaoRecall(False,5)

Recall: 0.6101694915254238
Duração: 11.110809


In [16]:
avaliacaoRecall(False,10)

Recall: 0.6677966101694915
Duração: 10.396010


In [17]:
avaliacaoRecall(False,20)

Recall: 0.7016949152542373
Duração: 11.523417


### Precision

In [18]:
avaliacaoPrecision(False,5)

Precision: 0.122034


In [19]:
avaliacaoPrecision(False,10)

Precision: 0.066780


In [20]:
avaliacaoPrecision(False,20)

Precision: 0.035085


### MAP

In [21]:
avaliacaoMAP(False,5)

MAP@ 5: 0.004146


In [22]:
avaliacaoMAP(False,10)

MAP@ 10: 0.003050


In [23]:
avaliacaoMAP(False,20)

MAP@ 20: 0.002415


### RR

In [24]:
avaliacaoRR(False,5)

RR: 0.389831


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [25]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [26]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [word for word in terms if word not in stopwords]
        terms = " ".join(terms)
        return terms
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [27]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_5_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [28]:
retriever = BM25Retriever(document_store=document_store)

In [29]:
pre = PreProcessamento()

In [30]:
pipeline = Pipeline()

In [31]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [32]:
avaliacaoRecall(True,5)

Recall: 0.6135593220338983
Duração: 6.319998


In [33]:
avaliacaoRecall(True,10)

Recall: 0.6644067796610169
Duração: 8.899968


In [34]:
avaliacaoRecall(True,20)

Recall: 0.7050847457627119
Duração: 8.763332


### Precision

In [35]:
avaliacaoPrecision(True,5)

Precision: 0.122712


In [36]:
avaliacaoPrecision(True,10)

Precision: 0.066441


In [37]:
avaliacaoPrecision(True,20)

Precision: 0.035254


### MAP

In [38]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003567


In [39]:
avaliacaoMAP(True,10)

MAP@ 10: 0.002488


In [40]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001845


### RR

In [41]:
avaliacaoRR(True,5)

RR: 0.379661


# Stemming

## 8- Stemming (Savoy)

In [42]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [43]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [44]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [45]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_8_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [46]:
retriever = BM25Retriever(document_store=document_store)

In [47]:
pre = PreProcessamento()

In [48]:
pipeline = Pipeline()

In [49]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [50]:
avaliacaoRecall(True,5)

Recall: 0.4745762711864407
Duração: 10.940616


In [51]:
avaliacaoRecall(True,10)

Recall: 0.5491525423728814
Duração: 11.929490


In [52]:
avaliacaoRecall(True,20)

Recall: 0.6135593220338983
Duração: 16.984783


### Precision

In [53]:
avaliacaoPrecision(True,5)

Precision: 0.094915


In [54]:
avaliacaoPrecision(True,10)

Precision: 0.054915


In [55]:
avaliacaoPrecision(True,20)

Precision: 0.030678


### MAP

In [56]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001685


In [57]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001176


In [58]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000648


### RR

In [59]:
avaliacaoRR(True,5)

RR: 0.301695


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [60]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [61]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [62]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_9_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [63]:
retriever = BM25Retriever(document_store=document_store)

In [64]:
pre = PreProcessamento()

In [65]:
pipeline = Pipeline()

In [66]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [67]:
avaliacaoRecall(True,5)

Recall: 0.5457627118644067
Duração: 6.604143


In [68]:
avaliacaoRecall(True,10)

Recall: 0.5830508474576271
Duração: 9.431793


In [69]:
avaliacaoRecall(True,20)

Recall: 0.6576271186440678
Duração: 12.889215


### Precision

In [70]:
avaliacaoPrecision(True,5)

Precision: 0.109153


In [71]:
avaliacaoPrecision(True,10)

Precision: 0.058305


In [72]:
avaliacaoPrecision(True,20)

Precision: 0.032881


### MAP

In [73]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002251


In [74]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001243


In [75]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001020


### RR

In [76]:
avaliacaoRR(True,5)

RR: 0.345763


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [77]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [78]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
    
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [79]:
  document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_18_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [80]:
retriever = BM25Retriever(document_store=document_store)

In [81]:
pre = PreProcessamento()

In [82]:
pipeline = Pipeline()

In [83]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [84]:
avaliacaoRecall(True,5)

Recall: 0.5627118644067797
Duração: 12.123640


In [85]:
avaliacaoRecall(True,10)

Recall: 0.6
Duração: 14.499931


In [86]:
avaliacaoRecall(True,20)

Recall: 0.6542372881355932
Duração: 24.119757


### Precision

In [87]:
avaliacaoPrecision(True,5)

Precision: 0.112542


In [88]:
avaliacaoPrecision(True,10)

Precision: 0.060000


In [89]:
avaliacaoPrecision(True,20)

Precision: 0.032712


### MAP

In [90]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002245


In [91]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001324


In [92]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000723


### RR

In [93]:
avaliacaoRR(True,5)

RR: 0.345763


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [94]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [95]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [96]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_21_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [97]:
retriever = BM25Retriever(document_store=document_store)

In [98]:
pre = PreProcessamento()

In [99]:
pipeline = Pipeline()

In [100]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [101]:
avaliacaoRecall(True,5)

Recall: 0.5830508474576271
Duração: 7.161255


In [102]:
avaliacaoRecall(True,10)

Recall: 0.6338983050847458
Duração: 7.579061


In [103]:
avaliacaoRecall(True,20)

Recall: 0.6779661016949152
Duração: 8.480478


### Precision

In [104]:
avaliacaoPrecision(True,5)

Precision: 0.116610


In [105]:
avaliacaoPrecision(True,10)

Precision: 0.063390


In [106]:
avaliacaoPrecision(True,20)

Precision: 0.033898


### MAP

In [107]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002145


In [108]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001354


In [109]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000735


### RR

In [110]:
avaliacaoRR(True,5)

RR: 0.372881


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLPS)

In [111]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [112]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [113]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_22_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [114]:
retriever = BM25Retriever(document_store=document_store)

In [115]:
pre = PreProcessamento()

In [116]:
pipeline = Pipeline()

In [117]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [118]:
avaliacaoRecall(True,5)

Recall: 0.6169491525423729
Duração: 5.740044


In [119]:
avaliacaoRecall(True,10)

Recall: 0.6508474576271186
Duração: 6.135203


In [120]:
avaliacaoRecall(True,20)

Recall: 0.7084745762711865
Duração: 6.809492


### Precision

In [121]:
avaliacaoPrecision(True,5)

Precision: 0.123390


In [122]:
avaliacaoPrecision(True,10)

Precision: 0.065085


In [123]:
avaliacaoPrecision(True,20)

Precision: 0.035424


### MAP

In [124]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003034


In [125]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001988


In [126]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001387


### RR

In [127]:
avaliacaoRR(True,5)

RR: 0.372881


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLPS) + unigram + bigram

In [128]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [129]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [130]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_23_img",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [131]:
retriever = BM25Retriever(document_store=document_store)

In [132]:
pre = PreProcessamento()

In [133]:
pipeline = Pipeline()

In [134]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [135]:
avaliacaoRecall(True,5)

Recall: 0.6
Duração: 7.385643


In [136]:
avaliacaoRecall(True,10)

Recall: 0.6508474576271186
Duração: 8.071139


In [137]:
avaliacaoRecall(True,20)

Recall: 0.6915254237288135
Duração: 8.881361


### Precision

In [138]:
avaliacaoPrecision(True,5)

Precision: 0.120000


In [139]:
avaliacaoPrecision(True,10)

Precision: 0.065085


In [140]:
avaliacaoPrecision(True,20)

Precision: 0.034576


### MAP

In [141]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002188


In [142]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001315


In [143]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000695


### RR

In [144]:
avaliacaoRR(True,5)

RR: 0.386441


# Indexacão

# Pré processamento

## 1- Sem pré processamento

In [145]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="sem_preprocessamento_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [146]:
retriever = BM25Retriever(document_store=document_store)

In [147]:
pipeline = Pipeline()

In [148]:
pipeline.add_node(component=retriever, name="Retriever",inputs=['Query'])

## Avaliação

### Recall

In [149]:
avaliacaoRecall(False,5)

Recall: 0.2135593220338983
Duração: 2.350755


In [150]:
avaliacaoRecall(False,10)

Recall: 0.2677966101694915
Duração: 2.501412


In [151]:
avaliacaoRecall(False,20)

Recall: 0.30847457627118646
Duração: 2.766187


### Precision

In [152]:
avaliacaoPrecision(False,5)

Precision: 0.042712


In [153]:
avaliacaoPrecision(False,10)

Precision: 0.026780


In [154]:
avaliacaoPrecision(False,20)

Precision: 0.015424


### MAP

In [155]:
avaliacaoMAP(False,5)

MAP@ 5: 0.000430


In [156]:
avaliacaoMAP(False,10)

MAP@ 10: 0.000404


In [157]:
avaliacaoMAP(False,20)

MAP@ 20: 0.000252


### RR

In [158]:
avaliacaoRR(False,5)

RR: 0.111864


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [159]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [160]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [word for word in terms if word not in stopwords]
        terms = " ".join(terms)
        return terms
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [161]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_5_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [162]:
retriever = BM25Retriever(document_store=document_store)

In [163]:
pre = PreProcessamento()

In [164]:
pipeline = Pipeline()

In [165]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [166]:
avaliacaoRecall(True,5)

Recall: 0.31864406779661014
Duração: 1.578219


In [167]:
avaliacaoRecall(True,10)

Recall: 0.3525423728813559
Duração: 1.698661


In [168]:
avaliacaoRecall(True,20)

Recall: 0.42033898305084744
Duração: 1.913376


### Precision

In [169]:
avaliacaoPrecision(True,5)

Precision: 0.063729


In [170]:
avaliacaoPrecision(True,10)

Precision: 0.035254


In [171]:
avaliacaoPrecision(True,20)

Precision: 0.021017


### MAP

In [172]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001034


In [173]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000687


In [174]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000406


### RR

In [175]:
avaliacaoRR(True,5)

RR: 0.169492


# Stemming

## 8- Stemming (Savoy)

In [176]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [177]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [178]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [179]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_8_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [180]:
retriever = BM25Retriever(document_store=document_store)

In [181]:
pre = PreProcessamento()

In [182]:
pipeline = Pipeline()

In [183]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [184]:
avaliacaoRecall(True,5)

Recall: 0.29152542372881357
Duração: 2.726175


In [185]:
avaliacaoRecall(True,10)

Recall: 0.34915254237288135
Duração: 2.944405


In [186]:
avaliacaoRecall(True,20)

Recall: 0.4033898305084746
Duração: 3.034096


### Precision

In [187]:
avaliacaoPrecision(True,5)

Precision: 0.058305


In [188]:
avaliacaoPrecision(True,10)

Precision: 0.034915


In [189]:
avaliacaoPrecision(True,20)

Precision: 0.020169


### MAP

In [190]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001218


In [191]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000719


In [192]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000443


### RR

In [193]:
avaliacaoRR(True,5)

RR: 0.176271


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [194]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [195]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [196]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_9_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [197]:
retriever = BM25Retriever(document_store=document_store)

In [198]:
pre = PreProcessamento()

In [199]:
pipeline = Pipeline()

In [200]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [201]:
avaliacaoRecall(True,5)

Recall: 0.33220338983050846
Duração: 1.761130


In [202]:
avaliacaoRecall(True,10)

Recall: 0.38305084745762713
Duração: 1.942361


In [203]:
avaliacaoRecall(True,20)

Recall: 0.4440677966101695
Duração: 2.460926


### Precision

In [204]:
avaliacaoPrecision(True,5)

Precision: 0.066441


In [205]:
avaliacaoPrecision(True,10)

Precision: 0.038305


In [206]:
avaliacaoPrecision(True,20)

Precision: 0.022203


### MAP

In [207]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001058


In [208]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000751


In [209]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000463


### RR

In [210]:
avaliacaoRR(True,5)

RR: 0.189831


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [211]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [212]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
    
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [213]:
  document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_18_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [214]:
retriever = BM25Retriever(document_store=document_store)

In [215]:
pre = PreProcessamento()

In [216]:
pipeline = Pipeline()

In [217]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [218]:
avaliacaoRecall(True,5)

Recall: 0.28135593220338984
Duração: 3.904474


In [219]:
avaliacaoRecall(True,10)

Recall: 0.3254237288135593
Duração: 5.641844


In [220]:
avaliacaoRecall(True,20)

Recall: 0.3898305084745763
Duração: 4.219683


### Precision

In [221]:
avaliacaoPrecision(True,5)

Precision: 0.056271


In [222]:
avaliacaoPrecision(True,10)

Precision: 0.032542


In [223]:
avaliacaoPrecision(True,20)

Precision: 0.019492


### MAP

In [224]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001126


In [225]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000652


In [226]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000398


### RR

In [227]:
avaliacaoRR(True,5)

RR: 0.159322


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [228]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [229]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [230]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_21_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [231]:
retriever = BM25Retriever(document_store=document_store)

In [232]:
pre = PreProcessamento()

In [233]:
pipeline = Pipeline()

In [234]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [235]:
avaliacaoRecall(True,5)

Recall: 0.27796610169491526
Duração: 2.631889


In [236]:
avaliacaoRecall(True,10)

Recall: 0.3423728813559322
Duração: 2.990520


In [237]:
avaliacaoRecall(True,20)

Recall: 0.4101694915254237
Duração: 3.567929


### Precision

In [238]:
avaliacaoPrecision(True,5)

Precision: 0.055593


In [239]:
avaliacaoPrecision(True,10)

Precision: 0.034237


In [240]:
avaliacaoPrecision(True,20)

Precision: 0.020508


### MAP

In [241]:
avaliacaoMAP(True,5)

MAP@ 5: 0.000812


In [242]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000524


In [243]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000378


### RR

In [244]:
avaliacaoRR(True,5)

RR: 0.152542


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLPS)

In [245]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [246]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [247]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_22_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [248]:
retriever = BM25Retriever(document_store=document_store)

In [249]:
pre = PreProcessamento()

In [250]:
pipeline = Pipeline()

In [251]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [252]:
avaliacaoRecall(True,5)

Recall: 0.4
Duração: 1.717355


In [253]:
avaliacaoRecall(True,10)

Recall: 0.4406779661016949
Duração: 1.826344


In [254]:
avaliacaoRecall(True,20)

Recall: 0.5050847457627119
Duração: 2.049564


### Precision

In [255]:
avaliacaoPrecision(True,5)

Precision: 0.080000


In [256]:
avaliacaoPrecision(True,10)

Precision: 0.044068


In [257]:
avaliacaoPrecision(True,20)

Precision: 0.025254


### MAP

In [258]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001214


In [259]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000858


In [260]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000481


### RR

In [261]:
avaliacaoRR(True,5)

RR: 0.230508


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLPS) + unigram + bigram

In [262]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [263]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [264]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_23_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [265]:
retriever = BM25Retriever(document_store=document_store)

In [266]:
pre = PreProcessamento()

In [267]:
pipeline = Pipeline()

In [268]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [269]:
avaliacaoRecall(True,5)

Recall: 0.34915254237288135
Duração: 2.245774


In [270]:
avaliacaoRecall(True,10)

Recall: 0.4033898305084746
Duração: 2.491656


In [271]:
avaliacaoRecall(True,20)

Recall: 0.45084745762711864
Duração: 3.311332


### Precision

In [272]:
avaliacaoPrecision(True,5)

Precision: 0.069831


In [273]:
avaliacaoPrecision(True,10)

Precision: 0.040339


In [274]:
avaliacaoPrecision(True,20)

Precision: 0.022542


### MAP

In [275]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001060


In [276]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000731


In [277]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000446


### RR

In [278]:
avaliacaoRR(True,5)

RR: 0.183051


# Emenda

# Pré processamento

## 1- Sem pré processamento

In [279]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="sem_preprocessamento_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [280]:
retriever = BM25Retriever(document_store=document_store)

In [281]:
pipeline = Pipeline()

In [282]:
pipeline.add_node(component=retriever, name="Retriever",inputs=['Query'])

## Avaliação

### Recall

In [283]:
avaliacaoRecall(False,5)

Recall: 0.3864406779661017
Duração: 3.666599


In [284]:
avaliacaoRecall(False,10)

Recall: 0.43728813559322033
Duração: 5.541443


In [285]:
avaliacaoRecall(False,20)

Recall: 0.4711864406779661
Duração: 4.338825


### Precision

In [286]:
avaliacaoPrecision(False,5)

Precision: 0.077288


In [287]:
avaliacaoPrecision(False,10)

Precision: 0.043729


In [288]:
avaliacaoPrecision(False,20)

Precision: 0.023559


### MAP

In [289]:
avaliacaoMAP(False,5)

MAP@ 5: 0.001689


In [290]:
avaliacaoMAP(False,10)

MAP@ 10: 0.000939


In [291]:
avaliacaoMAP(False,20)

MAP@ 20: 0.000493


### RR

In [292]:
avaliacaoRR(False,5)

RR: 0.244068


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [293]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [294]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [word for word in terms if word not in stopwords]
        terms = " ".join(terms)
        return terms
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [295]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_5_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [296]:
retriever = BM25Retriever(document_store=document_store)

In [297]:
pre = PreProcessamento()

In [298]:
pipeline = Pipeline()

In [299]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [300]:
avaliacaoRecall(True,5)

Recall: 0.4
Duração: 1.696531


In [301]:
avaliacaoRecall(True,10)

Recall: 0.4406779661016949
Duração: 1.790010


In [302]:
avaliacaoRecall(True,20)

Recall: 0.488135593220339
Duração: 1.992247


### Precision

In [303]:
avaliacaoPrecision(True,5)

Precision: 0.080000


In [304]:
avaliacaoPrecision(True,10)

Precision: 0.044068


In [305]:
avaliacaoPrecision(True,20)

Precision: 0.024407


### MAP

In [306]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001732


In [307]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000951


In [308]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000500


### RR

In [309]:
avaliacaoRR(True,5)

RR: 0.267797


# Stemming

## 8- Stemming (Savoy)

In [310]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [311]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [312]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [313]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_8_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [314]:
retriever = BM25Retriever(document_store=document_store)

In [315]:
pre = PreProcessamento()

In [316]:
pipeline = Pipeline()

In [317]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [318]:
avaliacaoRecall(True,5)

Recall: 0.3694915254237288
Duração: 2.788079


In [319]:
avaliacaoRecall(True,10)

Recall: 0.4135593220338983
Duração: 2.926431


In [320]:
avaliacaoRecall(True,20)

Recall: 0.4610169491525424
Duração: 3.081398


### Precision

In [321]:
avaliacaoPrecision(True,5)

Precision: 0.073898


In [322]:
avaliacaoPrecision(True,10)

Precision: 0.041356


In [323]:
avaliacaoPrecision(True,20)

Precision: 0.023051


### MAP

In [324]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001774


In [325]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000962


In [326]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000525


### RR

In [327]:
avaliacaoRR(True,5)

RR: 0.227119


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [328]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [329]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [330]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_9_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [331]:
retriever = BM25Retriever(document_store=document_store)

In [332]:
pre = PreProcessamento()

In [333]:
pipeline = Pipeline()

In [334]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [335]:
avaliacaoRecall(True,5)

Recall: 0.39322033898305087
Duração: 1.847597


In [336]:
avaliacaoRecall(True,10)

Recall: 0.44745762711864406
Duração: 1.964634


In [337]:
avaliacaoRecall(True,20)

Recall: 0.4711864406779661
Duração: 2.306371


### Precision

In [338]:
avaliacaoPrecision(True,5)

Precision: 0.078644


In [339]:
avaliacaoPrecision(True,10)

Precision: 0.044746


In [340]:
avaliacaoPrecision(True,20)

Precision: 0.023559


### MAP

In [341]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001828


In [342]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001017


In [343]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000537


### RR

In [344]:
avaliacaoRR(True,5)

RR: 0.261017


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [345]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [346]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
    
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [347]:
document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_18_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [348]:
retriever = BM25Retriever(document_store=document_store)

In [349]:
pre = PreProcessamento()

In [350]:
pipeline = Pipeline()

In [351]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [352]:
avaliacaoRecall(True,5)

Recall: 0.38305084745762713
Duração: 4.195381


In [353]:
avaliacaoRecall(True,10)

Recall: 0.43050847457627117
Duração: 4.440234


In [354]:
avaliacaoRecall(True,20)

Recall: 0.4847457627118644
Duração: 4.466233


### Precision

In [355]:
avaliacaoPrecision(True,5)

Precision: 0.076610


In [356]:
avaliacaoPrecision(True,10)

Precision: 0.043051


In [357]:
avaliacaoPrecision(True,20)

Precision: 0.024237


### MAP

In [358]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001833


In [359]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001011


In [360]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000566


### RR

In [361]:
avaliacaoRR(True,5)

RR: 0.244068


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [362]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [363]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [364]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_21_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [365]:
retriever = BM25Retriever(document_store=document_store)

In [366]:
pre = PreProcessamento()

In [367]:
pipeline = Pipeline()

In [368]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [369]:
avaliacaoRecall(True,5)

Recall: 0.39661016949152544
Duração: 2.797016


In [370]:
avaliacaoRecall(True,10)

Recall: 0.43389830508474575
Duração: 2.866767


In [371]:
avaliacaoRecall(True,20)

Recall: 0.48135593220338985
Duração: 3.547949


### Precision

In [372]:
avaliacaoPrecision(True,5)

Precision: 0.079322


In [373]:
avaliacaoPrecision(True,10)

Precision: 0.043390


In [374]:
avaliacaoPrecision(True,20)

Precision: 0.024068


### MAP

In [375]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001758


In [376]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000978


In [377]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000540


### RR

In [378]:
avaliacaoRR(True,5)

RR: 0.261017


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLPS)

In [379]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [380]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [381]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_22_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [382]:
retriever = BM25Retriever(document_store=document_store)

In [383]:
pre = PreProcessamento()

In [384]:
pipeline = Pipeline()

In [385]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [386]:
avaliacaoRecall(True,5)

Recall: 0.4271186440677966
Duração: 1.772321


In [387]:
avaliacaoRecall(True,10)

Recall: 0.488135593220339
Duração: 1.899911


In [388]:
avaliacaoRecall(True,20)

Recall: 0.5186440677966102
Duração: 2.125612


### Precision

In [389]:
avaliacaoPrecision(True,5)

Precision: 0.085424


In [390]:
avaliacaoPrecision(True,10)

Precision: 0.048814


In [391]:
avaliacaoPrecision(True,20)

Precision: 0.025932


### MAP

In [392]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001772


In [393]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001055


In [394]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000554


### RR

In [395]:
avaliacaoRR(True,5)

RR: 0.281356


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLPS) + unigram + bigram

In [396]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [397]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [398]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_23_emenda",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [399]:
retriever = BM25Retriever(document_store=document_store)

In [400]:
pre = PreProcessamento()

In [401]:
pipeline = Pipeline()

In [402]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [403]:
avaliacaoRecall(True,5)

Recall: 0.423728813559322
Duração: 2.455121


In [404]:
avaliacaoRecall(True,10)

Recall: 0.4711864406779661
Duração: 2.646664


In [405]:
avaliacaoRecall(True,20)

Recall: 0.5084745762711864
Duração: 3.285854


### Precision

In [406]:
avaliacaoPrecision(True,5)

Precision: 0.084746


In [407]:
avaliacaoPrecision(True,10)

Precision: 0.047119


In [408]:
avaliacaoPrecision(True,20)

Precision: 0.025424


### MAP

In [409]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001762


In [410]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001032


In [411]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000543


### RR

In [412]:
avaliacaoRR(True,5)

RR: 0.277966


# Emenda + Indexação

# Pré processamento

## 1- Sem pré processamento

In [413]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="sem_preprocessamento_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [414]:
retriever = BM25Retriever(document_store=document_store)

In [415]:
pipeline = Pipeline()

In [416]:
pipeline.add_node(component=retriever, name="Retriever",inputs=['Query'])

## Avaliação

### Recall

In [417]:
avaliacaoRecall(False,5)

Recall: 0.43050847457627117
Duração: 4.094975


In [418]:
avaliacaoRecall(False,10)

Recall: 0.5016949152542373
Duração: 4.364391


In [419]:
avaliacaoRecall(False,20)

Recall: 0.5491525423728814
Duração: 4.827554


### Precision

In [420]:
avaliacaoPrecision(False,5)

Precision: 0.086102


In [421]:
avaliacaoPrecision(False,10)

Precision: 0.050169


In [422]:
avaliacaoPrecision(False,20)

Precision: 0.027458


### MAP

In [423]:
avaliacaoMAP(False,5)

MAP@ 5: 0.001758


In [424]:
avaliacaoMAP(False,10)

MAP@ 10: 0.001104


In [425]:
avaliacaoMAP(False,20)

MAP@ 20: 0.000621


### RR

In [426]:
avaliacaoRR(False,5)

RR: 0.244068


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [427]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [428]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [word for word in terms if word not in stopwords]
        terms = " ".join(terms)
        return terms
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [429]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_5_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [430]:
retriever = BM25Retriever(document_store=document_store)

In [431]:
pre = PreProcessamento()

In [432]:
pipeline = Pipeline()

In [433]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [434]:
avaliacaoRecall(True,5)

Recall: 0.4406779661016949
Duração: 1.937387


In [435]:
avaliacaoRecall(True,10)

Recall: 0.5220338983050847
Duração: 2.078382


In [436]:
avaliacaoRecall(True,20)

Recall: 0.5898305084745763
Duração: 2.343803


### Precision

In [437]:
avaliacaoPrecision(True,5)

Precision: 0.088136


In [438]:
avaliacaoPrecision(True,10)

Precision: 0.052203


In [439]:
avaliacaoPrecision(True,20)

Precision: 0.029492


### MAP

In [440]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001782


In [441]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001042


In [442]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000659


### RR

In [443]:
avaliacaoRR(True,5)

RR: 0.261017


# Stemming

## 8- Stemming (Savoy)

In [444]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [445]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [446]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [447]:
document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_8_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [448]:
retriever = BM25Retriever(document_store=document_store)

In [449]:
pre = PreProcessamento()

In [450]:
pipeline = Pipeline()

In [451]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [452]:
avaliacaoRecall(True,5)

Recall: 0.4033898305084746
Duração: 3.315403


In [453]:
avaliacaoRecall(True,10)

Recall: 0.4610169491525424
Duração: 3.266070


In [454]:
avaliacaoRecall(True,20)

Recall: 0.5220338983050847
Duração: 3.551873


### Precision

In [455]:
avaliacaoPrecision(True,5)

Precision: 0.080678


In [456]:
avaliacaoPrecision(True,10)

Precision: 0.046102


In [457]:
avaliacaoPrecision(True,20)

Precision: 0.026102


### MAP

In [458]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001844


In [459]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001203


In [460]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000646


### RR

In [461]:
avaliacaoRR(True,5)

RR: 0.264407


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [462]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [463]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [464]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_9_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [465]:
retriever = BM25Retriever(document_store=document_store)

In [466]:
pre = PreProcessamento()

In [467]:
pipeline = Pipeline()

In [468]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [469]:
avaliacaoRecall(True,5)

Recall: 0.4406779661016949
Duração: 2.150806


In [470]:
avaliacaoRecall(True,10)

Recall: 0.4847457627118644
Duração: 2.327866


In [471]:
avaliacaoRecall(True,20)

Recall: 0.535593220338983
Duração: 2.717614


### Precision

In [472]:
avaliacaoPrecision(True,5)

Precision: 0.088136


In [473]:
avaliacaoPrecision(True,10)

Precision: 0.048475


In [474]:
avaliacaoPrecision(True,20)

Precision: 0.026780


### MAP

In [475]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001916


In [476]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001101


In [477]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000642


### RR

In [478]:
avaliacaoRR(True,5)

RR: 0.288136


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [479]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [480]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLPStemmer()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
    
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [481]:
  document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_18_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [482]:
retriever = BM25Retriever(document_store=document_store)

In [483]:
pre = PreProcessamento()

In [484]:
pipeline = Pipeline()

In [485]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [486]:
avaliacaoRecall(True,5)

Recall: 0.41694915254237286
Duração: 4.795602


In [487]:
avaliacaoRecall(True,10)

Recall: 0.4542372881355932
Duração: 5.081614


In [488]:
avaliacaoRecall(True,20)

Recall: 0.5084745762711864
Duração: 5.015390


### Precision

In [489]:
avaliacaoPrecision(True,5)

Precision: 0.083390


In [490]:
avaliacaoPrecision(True,10)

Precision: 0.045424


In [491]:
avaliacaoPrecision(True,20)

Precision: 0.025424


### MAP

In [492]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002011


In [493]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001074


In [494]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000646


### RR

In [495]:
avaliacaoRR(True,5)

RR: 0.250847


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [496]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [497]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = Savoy()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [498]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_21_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [499]:
retriever = BM25Retriever(document_store=document_store)

In [500]:
pre = PreProcessamento()

In [501]:
pipeline = Pipeline()

In [502]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [503]:
avaliacaoRecall(True,5)

Recall: 0.43050847457627117
Duração: 3.115163


In [504]:
avaliacaoRecall(True,10)

Recall: 0.48135593220338985
Duração: 3.836871


In [505]:
avaliacaoRecall(True,20)

Recall: 0.5186440677966102
Duração: 3.971284


### Precision

In [506]:
avaliacaoPrecision(True,5)

Precision: 0.086102


In [507]:
avaliacaoPrecision(True,10)

Precision: 0.048136


In [508]:
avaliacaoPrecision(True,20)

Precision: 0.025932


### MAP

In [509]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001909


In [510]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001062


In [511]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000566


### RR

In [512]:
avaliacaoRR(True,5)

RR: 0.267797


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLPS)

In [513]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [514]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1

    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = self._remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
        return " ".join(terms)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [515]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_22_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [516]:
retriever = BM25Retriever(document_store=document_store)

In [517]:
pre = PreProcessamento()

In [518]:
pipeline = Pipeline()

In [519]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [520]:
avaliacaoRecall(True,5)

Recall: 0.488135593220339
Duração: 2.062332


In [521]:
avaliacaoRecall(True,10)

Recall: 0.5627118644067797
Duração: 2.212214


In [522]:
avaliacaoRecall(True,20)

Recall: 0.5932203389830508
Duração: 2.482816


### Precision

In [523]:
avaliacaoPrecision(True,5)

Precision: 0.097627


In [524]:
avaliacaoPrecision(True,10)

Precision: 0.056271


In [525]:
avaliacaoPrecision(True,20)

Precision: 0.029661


### MAP

In [526]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002071


In [527]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001163


In [528]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000691


### RR

In [529]:
avaliacaoRR(True,5)

RR: 0.288136


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLPS) + unigram + bigram

In [530]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [531]:
class PreProcessamento(BaseComponent):
    outgoing_edges = 1
    
    def _remove_acentos(self,txt):
        return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

    def preprocess(self,txt):
        txt = str(txt)
        txt = _remove_acentos(txt)
        stopwords = nltk.corpus.stopwords.words("portuguese")
        stopwords.extend(list(punctuation))

        stemmer = RSLP_S()
        tokenizer = RegexpTokenizer('\w+')
        terms = tokenizer.tokenize(txt.lower())
        terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
        ngram = []
        ngram_1 = list(ngrams(terms, 1))
        ngram_2 = list(ngrams(terms, 2))
        for w in ngram_1:
            ngram.append(w[0])
        
        for w in ngram_2:
            string = w[0] + "_" + w[1]
            ngram.append(string)
    
        return " ".join(ngram)
      
    def run(
        self,
        query: Optional[str] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[MultiLabel] = None,
        documents: Optional[List[Document]] = None,
        meta: Optional[dict] = None,
    ) -> Tuple[Dict, str]:
        query = self.preprocess(query)
        output = {"query": query}
        return output, "output_1"

    def run_batch(
        self,
        queries: Optional[Union[str, List[str]]] = None,
        file_paths: Optional[List[str]] = None,
        labels: Optional[Union[MultiLabel, List[MultiLabel]]] = None,
        documents: Optional[Union[List[Document], List[List[Document]]]] = None,
        meta: Optional[Union[Dict[str, Any], List[Dict[str, Any]]]] = None,
        params: Optional[dict] = None,
        debug: Optional[bool] = None,
    ):
        pass


In [532]:
 document_store = ElasticsearchDocumentStore(host='localhost', port=9200, username='elastic',
                                                password='TlHbFKwz9YMAprROSrgA', index="preprocessamento_23_emenda_indexacao",
                                                search_fields=["content"])

This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.8.2. Unexpected behaviors or errors may occur due to version incompatibility.


In [533]:
retriever = BM25Retriever(document_store=document_store)

In [534]:
pre = PreProcessamento()

In [535]:
pipeline = Pipeline()

In [536]:
pipeline.add_node(component=pre, name="Pre",inputs=['Query'])
pipeline.add_node(component=retriever, name="Retriever",inputs=['Pre'])

## Avaliação

### Recall

In [537]:
avaliacaoRecall(True,5)

Recall: 0.4711864406779661
Duração: 2.712744


In [538]:
avaliacaoRecall(True,10)

Recall: 0.5084745762711864
Duração: 3.020604


In [539]:
avaliacaoRecall(True,20)

Recall: 0.5694915254237288
Duração: 4.029001


### Precision

In [540]:
avaliacaoPrecision(True,5)

Precision: 0.094237


In [541]:
avaliacaoPrecision(True,10)

Precision: 0.050847


In [542]:
avaliacaoPrecision(True,20)

Precision: 0.028475


### MAP

In [543]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001887


In [544]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001082


In [545]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000661


### RR

In [546]:
avaliacaoRR(True,5)

RR: 0.281356
